# Entity Matching with SDK-experimental demo

This notebook uses a small dummy data set to demonstrate how to do entity matching using cognite-sdk-python-experimental.

It aims at demonstrating most of the capabilities available, explain when the different parameter combinations are most suitable and explain (in some detail) what happens in the background.

## Get access to CDF
We assume you have some basic knowledge of CDF and the SDK. If not, please follow the 'lab' tutorials first.

To do this tutorial you nee have access to a Cognite project / tenant, you can apply for one here.


## Import modules
We need to import some Python modules in order to interact with CDF. We will use the Python SDK with Experimental Extensions, which we below refer to as a client.

In [ ]:
from cognite.client.exceptions import CogniteAPIError
from cognite.experimental import CogniteClient # version: ~0.22.3
from getpass import getpass
import pprint
pp = pprint.PrettyPrinter(indent=2)

## Create a client

To get access to your project, replace "yourproject" with your project name in the next cell. 

When you create the CogniteClient below, getpass will ask for your API key in an extra password field. Simply paste it in and press shift+enter.

In [ ]:
project = 'contextualization'
api_key = getpass("Please enter API key: ")
client = CogniteClient(project=project,
                       api_key=api_key,
                       client_name="dshub",
                      )

## Create dummy data

This tutorial uses a small dummy data set created below to demonstrate how to do entity matching using Python SDK with Experimental Extensions.

In [ ]:
match_from = [
    {"id":0, "name" : "KKL_21AA1019CA.PV", "description": "correct"}, 
    {"id":1, "name" : "KKL_13FV1234BU.VW", "description": "ok"}
]
match_to = [
    {"id":0, "name" : "21AA1019CA", "description": "correct"}, 
    {"id":1, "name" : "21AA1019CA", "description": "wrong"}, 
    {"id":2, "name" : "13FV1234BU"},
    {"id":3, "name" : "13FV1234BU", "description": "ok"}
]
true_matches = [(0,0)]

## Fit a supervised ml-model and predict for the same data

The supervised model calculates one or more similarity measures between match-to and match-from items. Then it uses these calculated similarity measures as features and fits a classification model using the labeled data.

Note, before calculating the similarity measures and training a model a set a candidate matches are selected. A pair of match-to and match-from items is considered to be a candidate if they have at least one token in common. Only the candidate match-from, match-to combinations are used in the training.  This is done to reduce computing time - calculating similarity measures for all possible combinations can be extremely heavy (10.000 time series and 30.000 assets -> 300.000.000 combinations). 

#### When to use a supervised ml-model?
A supervised ml-model is applicable when you have a number of labeled data. The more labeled data you have, the better results you might achieve. It is not recommended to apply for a supervised model if you have <500 labeled data.

In [ ]:
model = client.entity_matching.fit(match_from = match_from,
                                   match_to = match_to,
                                   true_matches = true_matches)

## Predict

When `predict` is called without any data, predictions are on the training data.

`num_matches` determines the number of matches to return for each `matchFrom` item, default is 1.

In [ ]:
job = model.predict(num_matches = 2)
matches = job.result
pp.pprint(matches["items"])

Note: For both `matchFrom` items we see that the two matches returned have an equal score. Hence, the model is not able to distinguish between the correct and incorrect match. 
Also, the scores are quite low. Unsupervised learning makes more sense when the data set is  small.

## Refit

Refit lets you retrain a model (using the same parameters) with additional labels/true-matches. The new `true_matches` (1,3) are added to the `true_matches`-list from the original model.   

To fit a model using only the (1,3) label. A new model must be trained using `fit`.

In [ ]:
model = model.refit(true_matches = [(1,3)])

In [ ]:
## Predict on training data
job = model.predict(num_matches = 2)
matches = job.result
pp.pprint(matches["items"])

Note: In this example the results are the same. The new true-match follows the exact same pattern as the original. 

## Fit unsupervised model

If there are no `true_matches` included in the `fit` call, an unsupervised model is trained.

As for a supervised model candidates are selected and similarity measures between the candidates are calculated. However, instead of training a classification model, the average of the average of the similarity measures are calculated and returned as the score. 

#### When to use a supervised ml-model?
When there are no or few true matches (labeled data), an unsupervised model is preferred.

In [ ]:
model = client.entity_matching.fit(match_from = match_from,
                                      match_to = match_to
) 

In [ ]:
# Predict on the training data
job = model.predict(num_matches = 2)
matches = job.result
pp.pprint(matches["items"])

Note: The scores have increased, but the model is still not able to distinguish between the correct and incorrect match. 

### Add additional keys
By default only name in `match_from` and name in `match_to` are used to calculate similarity measures. The `keys_from_to` parameter lets you specify all combinations of fields in `match_from` and `match_to` that should be used to calculate features.  

In this example it looks like also comparing the description field for both `match_to` and `match_from` will improve the model.

Note: Calculating similarity measures can be time consuming. Therefore, we avoid adding `keys_from_to` combinations which adds little or no information to the model. 

In [ ]:
try:
    model = client.entity_matching.fit(match_from = match_from,
                                       match_to = match_to,
                                       match_fields = [("name", "name")]
                                      )
except CogniteAPIError as error:
    print(error)

The request results in an error because one of the items in `match_to` is missing description. 
If the `complete_missing` is set to `True` missing values are replaced by empty strings.

### Add `ignore_missing_fields`

In [ ]:
model = client.entity_matching.fit(match_from = match_from,
                                   match_to = match_to,
                                   match_fields = [("name", "name"), ("description", "description")],
                                   ignore_missing_fields = True
)

In [ ]:
# Predict on training data
job = model.predict(num_matches = 2)
matches = job.result
pp.pprint(matches["items"])  

Note: The model now gives the correct matches a score of 1 and the incorrect matches score 0.5.

## Additional match_to items

The data below is the same as what was used in the previous examples, except that there are two new items in match_to.  
Id 10 and 13 are similar to 0 and 3 respectively, but the first letter combination ("AA" and "FV") are swapped with the prefix for the match_from items (KKL). 

We will now see how this leads to difficulties if we use the default `feature_type` ("simple”).

In [ ]:
match_from = [
    {"id":0, "name" : "KKL_21AA1019CA.PV", "description": "correct"}, 
    {"id":1, "name" : "KKL_13FV1234BU.VW", "description": "ok"}
]
match_to = [
    {"id":0,  "name" : "21AA1019CA", "description": "correct"},
    {"id":10, "name" : "21KKL1019CA", "description": "correct"},
    {"id":1,  "name" : "21AA1019CA", "description": "wrong"}, 
    {"id":2,  "name" : "13FV1234BU"},
    {"id":3,  "name" : "13FV1234BU", "description": "ok"},
    {"id":13, "name" : "13KKL1234BU", "description": "ok"}
]
true_matches = [(0,0), (1,3)]

In [ ]:
model = client.entity_matching.fit(match_from = match_from,
                                   match_to = match_to,
                                   match_fields = [("name", "name"), ("description", "description")],
                                   ignore_missing_fields = True,
) 

In [ ]:
# Predict on training data
job = model.predict(num_matches = 2)
matches = job.result
pp.pprint((matches["items"]))

Note: The new `match_to`-items have identical scores as the correct matches. 
This is because when using `feature_type`="simple" only the number of matching tokens are considered. 

Hence, for `match_to`-item with id 0 `21`, `AA`, `1019` and `CA` matches a token in `match-from`-item with id 0.
For `match_to`-item with id 10 `21`, `KKL`, `1019` and `CA` matches a token in `match-from`-item with id 0. Thus, the same number of tokens matches. 

The model does not take into account the `match_to`-item with id 0 have more and longer contiguous sequences of tokens.

The "bigram" `feature_type` does somewhat account for longer contiguous sequences of tokens. In addition to counting the number of matching tokens it also looks at the number of matching bigrams. That is, the number of matching tokens when two and two adjacent tokens are combined.

### Change `feature_type` 
The options of feature_type are:	"simple", "bigram", "frequency-weighted-bigram", "bigram-extra-tokenizers", "bigram-combo".

#### Simple
Simple is used by default for feature_type. This type is perfered to apply when one string is a substring of the other. For example BCDEF is a substring of A<u>BCDEF</u>G. 

#### Bigram
Adds similarity score based on the sequence of the terms. This type is applicable when two strings in the same or very similar sequences. For example, BCDEF and A<u>B</u>B<u>C</u><u>D</u><u>E</u>1<u>F</u>

In [ ]:
model = client.entity_matching.fit(match_from = match_from,
                                   match_to = match_to,
                                   match_fields = [("name", "name"), ("description", "description")],
                                   ignore_missing_fields = True,
                                   feature_type = "bigram"
)

In [ ]:
# Predict on training data
job = model.predict(num_matches = 2)
matches = job.result
pp.pprint((matches["items"]))

#### Frequency-Weighted-Bigram 
Calculates a similarity score based on the sequence of the terms, giving higher weights to less commonly occurring tokens.

In [ ]:
model = client.entity_matching.fit(match_from = match_from,
                                   match_to = match_to,
                                   match_fields = [("name", "name"), ("description", "description")],
                                   ignore_missing_fields = True,
                                   feature_type = "frequency-weighted-bigram"
)

In [ ]:
# Predict on training data
job = model.predict(num_matches = 2)
matches = job.result
pp.pprint((matches["items"]))

#### Bigram-Extra-Tokenizers
Similar to bigram, but able to learn that leading zeros and spaces should be ignored in matching. For example ABCDE and 000A<u>B</u>B<u>C</u><u>D</u><u>E</u>1<u>F</u>. Please note that this feature type could give score above 0.5 for two entity strings which are not match at all. 

In [ ]:
match_from = [
    {"id":0, "name" : "21AA1019CA", "description": "correct"},
    {"id":0, "name" : "J04_ONSTREAM_HOUR_AVG", "description": "correct"}
]
match_to = [
    {"id":0,  "name" : "0021AA1019CA", "description": "correct"},
    {"id":1,  "name" : "000000021AA1019CA", "description": "correct"},
    {"id":1,  "name" : "87-JB-004-J04", "description": "correct"}
]

model = client.entity_matching.fit(match_from = match_from,
                                   match_to = match_to,
                                   match_fields = [("name", "name"), ("description", "description")],
                                   ignore_missing_fields = True,
                                   feature_type = "bigram-extra-tokenizers"
)

In [ ]:
# Predict on training data
job = model.predict(num_matches = 2)
matches = job.result
pp.pprint((matches["items"]))

#### Bigram-Combo
Calculates all of the above options, relying on the model to determine the appropriate features to use. This is the slowest option.

In [ ]:
model = client.entity_matching.fit(match_from = match_from,
                                   match_to = match_to,
                                   match_fields = [("name", "name"), ("description", "description")],
                                   ignore_missing_fields = True,
                                   feature_type = "bigram-combo"
)

In [ ]:
# Predict on training data
job = model.predict(num_matches = 2)
matches = job.result
pp.pprint((matches["items"]))

## Predict on new data

To predict on new (unseen) data, simply add this data in the `predict`-call.

In [ ]:
match_from = [
    {"id":100, "name" : "KKL_44AB45", "description": "ok"}
]
match_to = [
    {"id":100,  "name" : "44AB45", "description": "ok"},
    {"id":101, "name" : "44AB45", "description": "ok12"},
    {"id":102,  "name" : "44AB45"}
]

job = model.predict(match_from = match_from,
                    match_to = match_to,
                    num_matches = 3,
                    )
matches = job.result
pp.pprint((matches["items"]))

## Get model info

If you have a model_id and want to know which parameters you used when training the model, use the `retrieve` method.

In [ ]:
client.entity_matching.retrieve(id = model.id)